In [1]:
import os
import copy
import time
import random
import string
import gc

# For data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

# Utils
from tqdm import tqdm
from collections import defaultdict

# Model Import
from ResNet20 import resnet20

In [2]:
CONFIG = {"seed": 42,
          "epochs": 200,
          "milestones":[100, 150],
          "train_batch_size": 128,
          "valid_batch_size": 256,
          "learning_rate": 0.1,
          "momentum": 0.9,
          "weight_decay": 1e-4,
          "num_classes": 10,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [3]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [4]:
'''def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(CONFIG['train_batch_size'])))'''

"def imshow(img):\n    img = img / 2 + 0.5     # unnormalize\n    npimg = img.numpy()\n    plt.imshow(np.transpose(npimg, (1, 2, 0)))\n    plt.show()\n\nclasses = ('plane', 'car', 'bird', 'cat',\n           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')\n\n# get some random training images\ndataiter = iter(trainloader)\nimages, labels = dataiter.next()\n\n# show images\nimshow(torchvision.utils.make_grid(images))\n# print labels\nprint(' '.join(f'{classes[labels[j]]:5s}' for j in range(CONFIG['train_batch_size'])))"

In [5]:
def criterion(outputs, labels):
    loss = nn.CrossEntropyLoss()
    return loss(outputs, labels)

In [6]:
def train_one_epoch(model, optimizer, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
            
        # рачсет вывода
        output = model(inputs)
        loss = criterion(output, targets)

        # расчет градиента
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_size = inputs.size(0) 
 
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix({'Epoch':epoch, 'Train_Loss':epoch_loss,
                            'LR':optimizer.param_groups[0]['lr']})
    gc.collect()
    return epoch_loss#, epoch_score


In [7]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    correct = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader), ncols=100)
    for step, data in bar:        
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
            
        # рачсет вывода
        output = model(inputs)
        loss = criterion(output, targets)
        
        _, preds = output.max(1)
        correct += preds.eq(targets).sum()

        batch_size = inputs.size(0)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        epoch_score = correct.cpu().detach().numpy() / dataset_size
        
        bar.set_postfix({'Epoch':epoch, 'Valid_Loss':epoch_loss, 'Valid_Score':epoch_score, 
                        'LR':optimizer.param_groups[0]['lr']})   
    
    gc.collect()
    return epoch_loss, epoch_score

In [8]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        scheduler.step()
        
        val_epoch_loss, val_epoch_score = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid Score'].append(val_epoch_score)
        
        print(f"Valid score {val_epoch_score}")
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            history["Best Loss"].append(best_epoch_loss)
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"ResNet20.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")

            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [9]:
def prepare_loaders():

    train_loader = torch.utils.data.DataLoader(
        torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.Pad(4),
            transforms.RandomCrop(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]), download=True),
        batch_size=CONFIG['train_batch_size'], shuffle=True,
        num_workers=2, pin_memory=True)

    valid_loader = torch.utils.data.DataLoader(
        torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])),
        batch_size=CONFIG['valid_batch_size'], shuffle=False,
        num_workers=2, pin_memory=True)
    
    return train_loader, valid_loader

In [10]:
# Create Dataloaders
train_loader, valid_loader = prepare_loaders()
    
model = resnet20()
model.to(CONFIG['device'])
    
# Определим Optimizer и Scheduler
    
optimizer = optim.SGD(model.parameters(), lr=CONFIG["learning_rate"], momentum=CONFIG["momentum"], weight_decay=CONFIG["weight_decay"])
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=CONFIG["milestones"])
    
model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'])
    
    
del model, history, train_loader, valid_loader
_ = gc.collect()
print()
    

Files already downloaded and verified
[INFO] Using GPU: NVIDIA GeForce RTX 2080



100%|███████████| 40/40 [00:00<00:00, 60.85it/s, Epoch=1, Valid_Loss=1.4, Valid_Score=0.493, LR=0.1]


Valid score 0.4929
Validation Loss Improved (inf ---> 1.402273502922058)
Model Saved



100%|██████████| 40/40 [00:00<00:00, 60.91it/s, Epoch=2, Valid_Loss=1.32, Valid_Score=0.577, LR=0.1]


Valid score 0.5773
Validation Loss Improved (1.402273502922058 ---> 1.324719365978241)
Model Saved



...

Valid score 0.922



100%|█████| 40/40 [00:00<00:00, 60.46it/s, Epoch=199, Valid_Loss=0.347, Valid_Score=0.922, LR=0.001]


Valid score 0.9218



100%|█████| 40/40 [00:00<00:00, 61.06it/s, Epoch=200, Valid_Loss=0.347, Valid_Score=0.923, LR=0.001]


Valid score 0.9231

Training complete in 0h 33m 17s
Best Loss: 0.2653

